In [1]:
from pyspark.sql import SparkSession

In [2]:
spark_create = SparkSession.builder.master("local").appName("Word Count")\
.config("spark.some.config.option", "some-value")\
.getOrCreate()

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("pyspark").getOrCreate()
sc=spark.sparkContext

spark

In [5]:
spark_create

In [6]:
sc

<SparkContext master=local appName=Word Count>

In [7]:
# in Python

df1 = spark.range(2, 10000000, 2)
df2 = spark.range(2, 10000000, 4)
step1 = df1.repartition(5)
step12 = df2.repartition(6)

step2 = step1.selectExpr("id * 5 as id")
step3 = step2.join(step12, ["id"])
step4 = step3.selectExpr("sum(id)")

step4.collect() # 2500000000000

[Row(sum(id)=2500000000000)]

In [8]:
step4.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(7) HashAggregate(keys=[], functions=[sum(id#8L)])
   +- ShuffleQueryStage 4
      +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=264]
         +- *(6) HashAggregate(keys=[], functions=[partial_sum(id#8L)])
            +- *(6) Project [id#8L]
               +- *(6) SortMergeJoin [id#8L], [id#2L], Inner
                  :- *(4) Sort [id#8L ASC NULLS FIRST], false, 0
                  :  +- AQEShuffleRead coalesced
                  :     +- ShuffleQueryStage 2
                  :        +- Exchange hashpartitioning(id#8L, 200), ENSURE_REQUIREMENTS, [plan_id=135]
                  :           +- *(3) Project [(id#0L * 5) AS id#8L]
                  :              +- ShuffleQueryStage 0
                  :                 +- Exchange RoundRobinPartitioning(5), REPARTITION_BY_NUM, [plan_id=68]
                  :                    +- *(1) Range (2, 10000000, step=2, splits=1)
                  +- *

In [9]:
spark